In [ ]:
using Pkg, Revise
Pkg.activate("../elementary-linear-algebra/GenLinAlgProblems")
using GenLinAlgProblems, LinearAlgebra, SparseArrays, BlockArrays, RowEchelon, Latexify, LaTeXStrings, SymPy, Random
using PyCall
;

<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">LaTeX Rendering Utilities for Julia</strong>
</div>

**Overview**

The **l_show*** functions allow easy display of julia scalars, vectors and matrices mixed with strings.
* *L_show*  produces a LaTeX representation of the arguments which could be copied into LaTeX documents
* *l_show*  uses *L_show* internally to render the resulting expressions in the julia notebook
* *py_show* uses *L_show* internally to render the resulting expressions in a python notebook

**Function Signature**
```julia
L_show(objs...; arraystyle=:parray, color=nothing, number_formatter=nothing,
       inline=true, factor_out=true, bold_matrix=false, per_element_style=nothing)
```
## **Arguments**
| **Argument**         | **Type**                     | **Description** |
|----------------------|----------------------------|----------------|
| `objs...`           | Any                         | Numbers, matrices, vectors, `BlockArray`, `SymPy` expressions, etc. |
| `arraystyle`        | `Symbol`                    | LaTeX matrix format (`:bmatrix`, `:pmatrix`, `:array`, etc.). |
| `color`             | `Union{Nothing, String}`    | Text color using `\textcolor{}` in LaTeX. |
| `number_formatter`  | `Union{Nothing, Function}`  | Function to format numbers before LaTeX conversion. |
| `factor_out`        | `Bool`                      | Factor out common denominators in rational entries. |
| `bold_matrix`       | `Bool`                      | Applies `\mathbf{}` to all matrix elements. |
| `per_element_style` | `Union{Nothing, Function}`  | Function `(x, i, j, formatted) -> styled_string` for per-element formatting. |
| `inline`            | `Bool`                      | If `true`, returns `$ ... $`; otherwise, `\[ ... \]`. |

# 1. Scalars

In [2]:
# Integers and Floats
display(l_show("The number 42 -> ",     42))                 # Expected: 42
display(l_show("The number 3.14 -> ", 3.14))                 # Expected: 3.14

# Rational Numbers
display(l_show("The fraction 1//3  -> ", 1//3))              # Expected: \frac{1}{3}

# Complex Numbers
display(l_show("The complex number 2+3i -> ", 2 + 3im))      # Expected: 2 + 3i

# Symbols and SymPy Symbols
display(l_show("The symbol :x  -> ", :x))                    # Expected: x
display(l_show("""The symbol SymPy.Sym("y") ->  """, SymPy.Sym("y"))) # Expected: y

# With Color and Formatting
display(l_show((text="1//3 colored red: ", color="black"), 1//3, color="red")) # Expected: \frac{1}{3}  colored red
display(l_show("42 bold -> ", 42, number_formatter=x -> "\\textbf{$x}"))       # Expected: bold 42

L"$\text{The number 42 -> } 42$
"

L"$\text{The number 3.14 -> } 3.14$
"

L"$\text{The fraction 1//3  -> } \frac{1}{3}$
"

L"$\text{The complex number 2+3i -> } 2 + 3\mathit{i}$
"

L"$\text{The symbol :x  -> } x$
"

L"$\text{The symbol SymPy.Sym(\"y\") ->  } y$
"

L"$\textcolor{black}{\text{1//3 colored red: }} \textcolor{red}{\frac{1}{3}}$
"

L"$\text{42 bold -> } \textbf{42}$
"

# 2. Strings and LaTeXStrings

In [3]:
display(l_show("Hello, World!"))                # Expected: \text{Hello, World!}
display(l_show("text_with_underscores"))        # Expected: \text{text\_with\_underscores}
display(l_show(L"A = B + C"))                   # Expected: A = B + C (keeps LaTeX raw)
display(l_show(L"A = B \cdot C", color="red"))  # Expected: \textcolor{red}{A = B \cdot C}

L"$\text{Hello, World!}$
"

L"$\text{text\_with\_underscores}$
"

L"$A = B + C$
"

L"$\textcolor{red}{A = B \cdot C}$
"

# 3. Vectors

In [4]:
# ✅ Empty vector
display(l_show("Empty Vector: ", Vector{Int}(), arraystyle=:bmatrix))             # Should print empty [ ]

A=[1,2,3im]
# ✅ Column vector
display(l_show("Column vector: A=", A, L"\quad A^T =", transpose(A), L"\quad A^H =", A', arraystyle=:bmatrix))

# ✅ Symbolic vector
A =  [Sym("x"), Sym("y"), 3im]
display(l_show("Symbolic Vector: ", A, L"\quad A^T =", transpose(A), L"\quad A^H =", A', arraystyle=:Bmatrix, bold_matrix=true))


L"$\text{Empty Vector: } \begin{bmatrix}

\end{bmatrix}$
"

L"$\text{Column vector: A=} \begin{bmatrix}
1 \\
2 \\
3\mathit{i} \\
\end{bmatrix} \quad A^T = \begin{bmatrix}
1 & 2 & 3\mathit{i} \\
\end{bmatrix} \quad A^H = \begin{bmatrix}
1 & 2 & -3\mathit{i} \\
\end{bmatrix}$
"

L"$\text{Symbolic Vector: } \mathbf{} \begin{Bmatrix}
\mathbf{x} \\
\mathbf{y} \\
\mathbf{3 \cdot I} \\
\end{Bmatrix} \quad A^T = \mathbf{} \begin{Bmatrix}
\mathbf{\mathrm{transpose}\left( x \right)} & \mathbf{\mathrm{transpose}\left( y \right)} & \mathbf{3 \cdot I} \\
\end{Bmatrix} \quad A^H = \mathbf{} \begin{Bmatrix}
\mathbf{\mathrm{conjugate}\left( x \right)} & \mathbf{\mathrm{conjugate}\left( y \right)} & \mathbf{-3 \cdot I} \\
\end{Bmatrix}$
"

# 4. Standard Matrices

In [5]:
A = [1 2; 3 4]
display(l_show("integer matrix A=", A))  # Expected: LaTeX bmatrix of [[1,2]; [3,4]]

B = [1.5 2.5; 3.5 4.5]
display(l_show("real matrix B=", B, arraystyle=:pmatrix))  # Expected: LaTeX pmatrix

C = [1//2 1//3; 1//4 1//5]
display(l_show("not factored C=", C, factor_out=false))     # Expected: Fraction elements in LaTeX
display(l_show("default: factored C=", C))                       # Expected: Fraction factored out

# Colored and formatted matrices
display(l_show("green with braces A=", (A, color="green", arraystyle=:Bmatrix)))  # Green curly braces matrix
display(l_show("bold C=", (C, bold_matrix=true)))               # Bold fractions

# number formatting
display(l_show(L"C \approx", (C, factor_out=false, number_formatter=x->round_value(x,2))))


L"$\text{integer matrix A=} \left(\begin{array}{rr}
1 & 2 \\
3 & 4 \\
\end{array}\right)$
"

L"$\text{real matrix B=} \begin{pmatrix}
1.5 & 2.5 \\
3.5 & 4.5 \\
\end{pmatrix}$
"

L"$\text{not factored C=} \left(\begin{array}{rr}
\frac{1}{2} & \frac{1}{3} \\
\frac{1}{4} & \frac{1}{5} \\
\end{array}\right)$
"

L"$\text{default: factored C=} \frac{1}{60} \left(\begin{array}{rr}
30 & 20 \\
15 & 12 \\
\end{array}\right)$
"

L"$\text{green with braces A=} \textcolor{green}{\begin{Bmatrix}
1 & 2 \\
3 & 4 \\
\end{Bmatrix}}$
"

L"$\text{bold C=} \mathbf{\frac{1}{60}} \left(\begin{array}{rr}
\mathbf{30} & \mathbf{20} \\
\mathbf{15} & \mathbf{12} \\
\end{array}\right)$
"

L"$C \approx \left(\begin{array}{rr}
0.5 & 0.33 \\
0.25 & 0.2 \\
\end{array}\right)$
"

# 5. Complex & Symbolic Matrices

In [6]:
D = [1 2im//3; (-1)//2 5//2im]
display(l_show("Complex Matrix D =", D, L"\quad D^T =", transpose(D), L"\qquad D^H =", D'))  # Expected: Matrix with complex numbers

E = [SymPy.Sym("x") 2; 3 SymPy.Sym("y")]
display(l_show("Matrix with symbols ", E))  # Expected: Matrix with symbols x and y

# Complex + Symbols
F = [1//2 SymPy.Sym("x"); (1+im)//3 2*SymPy.Sym("y")]
display(l_show("Complex matrix with symbols ", F))  # Expected: Mixed numeric & symbolic matrix

display(l_show("Factoring by hand... ",  1//6, 6F))


L"$\text{Complex Matrix D =} \frac{1}{6} \left(\begin{array}{rr}
6 & 4\mathit{i} \\
-3 & -15\mathit{i} \\
\end{array}\right) \quad D^T = \frac{1}{6} \left(\begin{array}{rr}
6 & -3 \\
4\mathit{i} & -15\mathit{i} \\
\end{array}\right) \qquad D^H = \frac{1}{6} \left(\begin{array}{rr}
6 & -3 \\
-4\mathit{i} & 15\mathit{i} \\
\end{array}\right)$
"

L"$\text{Matrix with symbols } \left(\begin{array}{rr}
x & 2 \\
3 & y \\
\end{array}\right)$
"

L"$\text{Complex matrix with symbols } \left(\begin{array}{rr}
\frac{1}{2} & x \\
\frac{1}{3} + \frac{I}{3} & 2 \cdot y \\
\end{array}\right)$
"

L"$\text{Factoring by hand... } \frac{1}{6} \left(\begin{array}{rr}
3 & 6 \cdot x \\
2 + 2 \cdot I & 12 \cdot y \\
\end{array}\right)$
"

# 6. Special Matrices in Julia

In [7]:
using SparseArrays

# Adjoint (Transpose) Matrix
A_adj = adjoint(A)
display(l_show("Transpose: ", A_adj))  # Expected: Transposed bmatrix

# Symmetric and Hermitian Matrices
S = Symmetric(A)
display(l_show("Symmetric ", S))  # Expected: Symmetric matrix

H = Hermitian(A)
display(l_show("Hermitian ", H))  # Expected: Hermitian matrix

# Diagonal Matrix
D = Diagonal([1,2,3])
display(l_show("Diagonal ", D))  # Expected: Diagonal matrix

# Sparse Matrix
S = sparse([1,2], [1,2], [10,20])
display(l_show("Sparse ", S))  # Expected: Sparse matrix converted to full matrix


L"$\text{Transpose: } \left(\begin{array}{rr}
1 & 3 \\
2 & 4 \\
\end{array}\right)$
"

L"$\text{Symmetric } \left(\begin{array}{rr}
1 & 2 \\
2 & 4 \\
\end{array}\right)$
"

L"$\text{Hermitian } \left(\begin{array}{rr}
1 & 2 \\
2 & 4 \\
\end{array}\right)$
"

L"$\text{Diagonal } \left(\begin{array}{rrr}
1 & 0 & 0 \\
0 & 2 & 0 \\
0 & 0 & 3 \\
\end{array}\right)$
"

L"$\text{Sparse } \left(\begin{array}{rr}
10 & 0 \\
0 & 20 \\
\end{array}\right)$
"

# 7. BlockArrays

In [8]:
A = BlockArray(
    [1 2 3; 4 5 6; 7 8 9],  # Regular matrix
    [1, 2],  # Row blocks: [1, 2]
    [1, 2]   # Column blocks: [1, 2]
)
l_show( "block array of integers A=", A, arraystyle=:bmatrix)

L"$\text{block array of integers A=} \left[\begin{array}{r|rr}
1 & 2 & 3 \\ \hline
4 & 5 & 6 \\
7 & 8 & 9 \\
\end{array}\right]$
"

In [9]:
C = BlockArray(
    fill(1.0, 4, 6),  # Regular matrix
    [2, 2],  # Row blocks: [2, 2]
    [3, 3]   # Column blocks: [3, 3]
)
l_show("block array of float, C=", C, number_formatter=x->round_value(x,0), arraystyle=:vmatrix)

L"$\text{block array of float, C=} \left|\begin{array}{rrr|rrr}
1 & 1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 & 1 \\ \hline
1 & 1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 & 1 \\
\end{array}\right|$
"

In [10]:
D = BlockArray(
    [1+im  2-im  3+2im;
     4-2im  5+im  6-im;
     7+im  8-im  9+2im],  # Complex integer matrix
    [1, 2],               # Row blocks: [1, 2]
    [1, 1, 1]             # Column blocks
)
l_show("blockarray of complex ints: D=", D)


L"$\text{blockarray of complex ints: D=} \left(\begin{array}{r|rr}
1 + \mathit{i} & 2 - \mathit{i} & 3 + 2\mathit{i} \\ \hline
4 - 2\mathit{i} & 5 + \mathit{i} & 6 - \mathit{i} \\
7 + \mathit{i} & 8 - \mathit{i} & 9 + 2\mathit{i} \\
\end{array}\right)$
"

In [11]:
E = BlockArray(
    [(1+1im//3)  ((2-1im)//2)  3+2im;
     4-2im  5//2+im  6-im//10;
     7+im//4  8-im  9+2im],  # Complex rational matrix
    [1, 2],  # Row blocks
    [1, 2]   # Column blocks
)
highlight = (x, i, j, formatted) -> i ==1 && j == 1 ? "\\textcolor{red}{{$formatted}}" : formatted
l_show(E, bold_matrix=true, per_element_style=highlight)

L"$\mathbf{\frac{1}{60}} \left(\begin{array}{r|rr}
\mathbf{\textcolor{red}{{60 + 20\mathit{i}}}} & \mathbf{60 - 30\mathit{i}} & \mathbf{180 + 120\mathit{i}} \\ \hline
\mathbf{240 - 120\mathit{i}} & \mathbf{150 + 60\mathit{i}} & \mathbf{360 - 6\mathit{i}} \\
\mathbf{420 + 15\mathit{i}} & \mathbf{480 - 60\mathit{i}} & \mathbf{540 + 120\mathit{i}} \\
\end{array}\right)$
"

In [12]:
A = BlockArray(
    [1 2 4; 3 4 5],  # Regular matrix
    [1, 1],      # Row blocks
    [2, 1]       # Column blocks
)

function color_odd_numbers(x, i, j, latex_str)
    return isodd(x) ? "\\textcolor{red}{$latex_str}" : latex_str
end

# Test L_show with per_element_style on BlockArray
display(l_show("Colorize odd values,  A=", A; per_element_style=color_odd_numbers))

L"$\text{Colorize odd values,  A=} \left(\begin{array}{rr|r}
\textcolor{red}{1} & 2 & 4 \\ \hline
\textcolor{red}{3} & 4 & \textcolor{red}{5} \\
\end{array}\right)$
"

# 8. Per Element Formatting

In [13]:
diagonal_highlight = (x, i, j, formatted) -> i == j ? "\\textcolor{red}{" * formatted * "}" : formatted
l_show((C, bold_matrix=true, per_element_style=diagonal_highlight))

L"$\mathbf{} \left(\begin{array}{rrr|rrr}
\mathbf{\textcolor{red}{1.0}} & \mathbf{1.0} & \mathbf{1.0} & \mathbf{1.0} & \mathbf{1.0} & \mathbf{1.0} \\
\mathbf{1.0} & \mathbf{\textcolor{red}{1.0}} & \mathbf{1.0} & \mathbf{1.0} & \mathbf{1.0} & \mathbf{1.0} \\ \hline
\mathbf{1.0} & \mathbf{1.0} & \mathbf{\textcolor{red}{1.0}} & \mathbf{1.0} & \mathbf{1.0} & \mathbf{1.0} \\
\mathbf{1.0} & \mathbf{1.0} & \mathbf{1.0} & \mathbf{\textcolor{red}{1.0}} & \mathbf{1.0} & \mathbf{1.0} \\
\end{array}\right)$
"

In [14]:
row_color = (x, i, j, formatted) -> i % 2 == 0 ? "\\boxed{\\textcolor{purple}{" * formatted * "}}" : formatted
P=[1 2 3 4 5; 6 7 8 9 10; -1 -2 -3 -4 -5; -6 -7 -8 -9 -10]
l_show(( P, per_element_style=row_color, arraystyle=:array))

L"$\begin{array}{rrrrr}
1 & 2 & 3 & 4 & 5 \\
\boxed{\textcolor{purple}{6}} & \boxed{\textcolor{purple}{7}} & \boxed{\textcolor{purple}{8}} & \boxed{\textcolor{purple}{9}} & \boxed{\textcolor{purple}{10}} \\
-1 & -2 & -3 & -4 & -5 \\
\boxed{\textcolor{purple}{-6}} & \boxed{\textcolor{purple}{-7}} & \boxed{\textcolor{purple}{-8}} & \boxed{\textcolor{purple}{-9}} & \boxed{\textcolor{purple}{-10}} \\
\end{array}$
"

# 9. Combining Different Types

In [15]:
display(l_show(
    "Equation:",
    (A, color="blue", arraystyle=:pmatrix),
    L"\ne",
    (B, color="red", arraystyle=:bmatrix),
    L";\qquad x =",
    (; value=1//3, color="green"),  # need a named tuple to apply options to a scalar
    L"\quad", "i.e., ", 1//3,
    color="purple"                  # global option
))


L"$\textcolor{purple}{\text{Equation:}} \textcolor{blue}{\left(\begin{array}{rr|r}
1 & 2 & 4 \\ \hline
3 & 4 & 5 \\
\end{array}\right)} \textcolor{purple}{\ne} \textcolor{red}{\begin{bmatrix}
1.5 & 2.5 \\
3.5 & 4.5 \\
\end{bmatrix}} \textcolor{purple}{;\qquad x =} \textcolor{green}{\frac{1}{3}} \textcolor{purple}{\quad} \textcolor{purple}{\text{i.e., }} \textcolor{purple}{\frac{1}{3}}$
"

# 10. Inline vs Block LaTeX

In [16]:
display(l_show("left justified: A = ", A, inline=true))   # Inline: Expected `$ \begin{bmatrix} ... \end{bmatrix} $`
display(l_show("centered: A = ", A, inline=false))  # Block: Expected `\[ \begin{bmatrix} ... \end{bmatrix} \]`


L"$\text{left justified: A = } \left(\begin{array}{rr|r}
1 & 2 & 4 \\ \hline
3 & 4 & 5 \\
\end{array}\right)$
"

L"\[\text{centered: A = } \left(\begin{array}{rr|r}
1 & 2 & 4 \\ \hline
3 & 4 & 5 \\
\end{array}\right)\]
"

# 11. Obtaining Raw LaTeX Representations

In [17]:
# print the output of L_show (capitalized) instead of l_show

println(L_show(
    "Equation:",
    (A, color="blue", arraystyle=:pmatrix),
    L"\ne",
    (B, color="red", arraystyle=:array),
    L";\qquad x =",
    (; value=1//3, color="green"),  # need a named tuple to apply options to a scalar
    L"\quad", "i.e., ", 1//3,
    color="purple"                  # global option
))

$\textcolor{purple}{\text{Equation:}} \textcolor{blue}{\left(\begin{array}{rr|r}
1 & 2 & 4 \\ \hline
3 & 4 & 5 \\
\end{array}\right)} \textcolor{purple}{\ne} \textcolor{red}{\begin{array}{rr}
1.5 & 2.5 \\
3.5 & 4.5 \\
\end{array}} \textcolor{purple}{;\qquad x =} \textcolor{green}{\frac{1}{3}} \textcolor{purple}{\quad} \textcolor{purple}{\text{i.e., }} \textcolor{purple}{\frac{1}{3}}$



# Appendix A: Additional Tests

In [18]:
function test_L_show()
    println("🚀 Running L_show Test Cases...\n")

    ## ✅ Case 1: Empty Matrix
    empty_mat = Matrix{Int}(undef, 0, 0)
    display(l_show("Test 1: Empty Matrix:   ", empty_mat, arraystyle=:bmatrix))

    ## ✅ Case 2: Single-Element Matrix
    single_element = [42]
    display(l_show("Test 2: Single-Element Matrix   ", single_element, arraystyle=:pmatrix))

    ## ✅ Case 3: Matrix with Symbols
    sym_mat = [Sym("x") Sym("y"); Sym("a") Sym("b")]
    display(l_show("Test 3: Matrix with Symbols   ", sym_mat, arraystyle=:Bmatrix))

    ## ✅ Case 4: Adjoint (Transpose) Matrices
    adjoint_mat = [1 2; 3 4]'
    display(l_show("Test 4: Adjoint (Transpose) Matrix   ", adjoint_mat, arraystyle=:bmatrix))

    ## ✅ Case 5: Diagonal Matrix
    diagonal_mat = Diagonal([1, 2, 3])
    display(l_show("Test 5: Diagonal Matrix   ", diagonal_mat, arraystyle=:bmatrix))

    ## ✅ Case 6: Sparse Matrix
    sparse_mat = sparse([1 0; 0 1])
    display(l_show("Test 6: Sparse Matrix   ", sparse_mat, arraystyle=:bmatrix))

    ## ✅ Case 7: Custom `per_element_style` (Row-Based Color)
    row_color = (x, i, j, formatted) -> i % 2 == 0 ? "\\textcolor{blue}{" * formatted * "}" : formatted
    println()
    display(l_show("Test 7: Row-Based Color   ", [1 2; 3 4], per_element_style=row_color, arraystyle=:bmatrix))

    ## ✅ Case 8: Highlighting the Diagonal
    diagonal_highlight = (x, i, j, formatted) -> i == j ? "\\textbf{" * formatted * "}" : formatted
    display(l_show("Test 8: Highlighting the Diagonal   ", [1 2; 3 4], per_element_style=diagonal_highlight, arraystyle=:bmatrix))

    ## ✅ Case 9: Factor Extraction
    frac_mat = [1//3 2//3; 4//9 5//9]
    display(l_show("Test 9: Factor Extraction  ", frac_mat, arraystyle=:bmatrix, factor_out=true))

    ## ✅ Case 10: Mixing Multiple Matrices
    display(l_show("Test 10: Multiple Matrices Side by Side   ", (matrix=[1 2; 3 -4], arraystyle=:array), [5 6; 7 8], arraystyle=:bmatrix))

    ## ✅ Case 11: Bold Matrix with Custom Styling
    display(l_show("Test 11: Bold Matrix with Custom Styling   ", [1 2; 3 4], bold_matrix=true, per_element_style=row_color, arraystyle=:bmatrix))
    println(L_show("Test 11: Bold Matrix with Custom Styling   ", [1 2; 3 4], bold_matrix=true, per_element_style=row_color, arraystyle=:bmatrix))
end
test_L_show()

🚀 Running L_show Test Cases...



L"$\text{Test 1: Empty Matrix:   } \begin{bmatrix}

\end{bmatrix}$
"

L"$\text{Test 2: Single-Element Matrix   } \begin{pmatrix}
42 \\
\end{pmatrix}$
"

L"$\text{Test 3: Matrix with Symbols   } \begin{Bmatrix}
x & y \\
a & b \\
\end{Bmatrix}$
"

L"$\text{Test 4: Adjoint (Transpose) Matrix   } \begin{bmatrix}
1 & 3 \\
2 & 4 \\
\end{bmatrix}$
"

...

L"$\text{Test 8: Highlighting the Diagonal   } \begin{bmatrix}
\textbf{1} & 2 \\
3 & \textbf{4} \\
\end{bmatrix}$
"

L"$\text{Test 9: Factor Extraction  } \frac{1}{9} \begin{bmatrix}
3 & 6 \\
4 & 5 \\
\end{bmatrix}$
"

L"$\text{Test 10: Multiple Matrices Side by Side   } \begin{array}{rr}
1 & 2 \\
3 & -4 \\
\end{array} \begin{bmatrix}
5 & 6 \\
7 & 8 \\
\end{bmatrix}$
"

L"$\text{Test 11: Bold Matrix with Custom Styling   } \mathbf{} \begin{bmatrix}
\mathbf{1} & \mathbf{2} \\
\mathbf{\textcolor{blue}{3}} & \mathbf{\textcolor{blue}{4}} \\
\end{bmatrix}$
"

$\text{Test 11: Bold Matrix with Custom Styling   } \mathbf{} \begin{bmatrix}
\mathbf{1} & \mathbf{2} \\
\mathbf{\textcolor{blue}{3}} & \mathbf{\textcolor{blue}{4}} \\
\end{bmatrix}$



In [19]:
function rand_complex_rational_matrix(m, n)
    real_part = rand(1:2, m, n) .// rand(1:3, m, n)  # Random rational numbers for real part
    imag_part = rand(1:2, m, n) .// rand(1:3, m, n)  # Random rational numbers for imaginary part

    return Complex{Rational{Int}}.(real_part, imag_part)  # Construct Complex{Rational{Int}}
end
A = rand_complex_rational_matrix(6,6)
B = BlockArray(A, [2, 4], [3, 3])  # Blocked matrix: 2 rows followed by 4 rows, 3 columns followed by 4 columns

display(l_show(" B =", B, L"\quad B^T=", transpose(B),
        arraystyle=:bmatrix))


"\$\\text{ B =} \\frac{1}{6} \\left[\\begin{array}{rrr|rrr}\n6 + 3\\mathit{i} & 3 + 2\\mathit{i} & 6 + 2\\mathit{i} & 12 + 6\\mathit{i} & 3 + 12\\mathit{i} & 3 + 2\\mathit{i} \\\\\n12 + 12\\mathit{i} & 4 + 3\\mathit{i} & 6 + 12\\mathit{i} & 3 + 12\\mathit{i} & 2 + 6\\mathit{i} & 3 + 6\\mathit" ⋯ 948 bytes ⋯ "it{i} & 12 + 4\\mathit{i} \\\\\n3 + 12\\mathit{i} & 2 + 6\\mathit{i} & 6 + 6\\mathit{i} & 6 + 3\\mathit{i} & 2 + 2\\mathit{i} & 4 + 4\\mathit{i} \\\\\n3 + 2\\mathit{i} & 3 + 6\\mathit{i} & 12 + 2\\mathit{i} & 12 + 6\\mathit{i} & 6 + 12\\mathit{i} & 6 + 4\\mathit{i} \\\\\n\\end{array}\\right]\$\n"

In [20]:
display(l_show(L"-i*B =", -1im*B, L",\quad B^H = ", B',  arraystyle=:bmatrix))

"\$-i*B = \\frac{1}{6} \\left[\\begin{array}{rrr|rrr}\n3 - 6\\mathit{i} & 2 - 3\\mathit{i} & 2 - 6\\mathit{i} & 6 - 12\\mathit{i} & 12 - 3\\mathit{i} & 2 - 3\\mathit{i} \\\\\n12 - 12\\mathit{i} & 3 - 4\\mathit{i} & 12 - 6\\mathit{i} & 12 - 3\\mathit{i} & 6 - 2\\mathit{i} & 6 - 3\\mathit{i} \\" ⋯ 946 bytes ⋯ "it{i} & 12 - 4\\mathit{i} \\\\\n3 - 12\\mathit{i} & 2 - 6\\mathit{i} & 6 - 6\\mathit{i} & 6 - 3\\mathit{i} & 2 - 2\\mathit{i} & 4 - 4\\mathit{i} \\\\\n3 - 2\\mathit{i} & 3 - 6\\mathit{i} & 12 - 2\\mathit{i} & 12 - 6\\mathit{i} & 6 - 12\\mathit{i} & 6 - 4\\mathit{i} \\\\\n\\end{array}\\right]\$\n"

In [21]:
function test_L_show(::Type{T}) where T
    println("\n🔹 Testing Standard Vector:")
    v = T.([1, 2, 3])  # Convert elements to type T
    display(l_show(v))

    println("\n🔹 Testing Adjoint Vector (Row Vector):")
    v_adj = v'
    display(l_show(v_adj))

    println("\n🔹 Testing Standard Matrix:")
    M = T.([1 2; 3 4])
    display(l_show(M))

    println("\n🔹 Testing Adjoint Matrix (Transposed Matrix):")
    M_adj = M'
    display(l_show(M_adj))

    println("\n🔹 Testing BlockArray:")
    A = T.(round.(rand(6, 6)))  # Convert random matrix to type T
    B = BlockArray(A, [2, 4], [3, 3])  # Partition into (2+4) × (3+3) blocks
    display(l_show(B))

    println("\n🔹 Testing Adjoint BlockArray:")
    B_adj = B'
    display(l_show(B_adj))

    println("\n✅ All tests completed for type: ", T)
end


exhaustive_test_L_show (generic function with 1 method)

In [22]:
exhaustive_test_L_show(Int)


🔹 Testing Standard Vector:


L"$\left(\begin{array}{r}
1 \\
2 \\
3 \\
\end{array}\right)$
"


🔹 Testing Adjoint Vector (Row Vector):


L"$\left(\begin{array}{r}
1 & 2 & 3 \\
\end{array}\right)$
"


🔹 Testing Standard Matrix:


...


🔹 Testing BlockArray:


L"$\left(\begin{array}{rrr|rrr}
1 & 1 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 1 & 1 & 1 \\ \hline
1 & 0 & 0 & 1 & 0 & 1 \\
1 & 1 & 1 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 1 \\
0 & 1 & 1 & 1 & 1 & 0 \\
\end{array}\right)$
"


🔹 Testing Adjoint BlockArray:


L"$\left(\begin{array}{rr|rrrr}
1 & 0 & 1 & 1 & 0 & 0 \\
1 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 & 1 \\ \hline
0 & 1 & 1 & 1 & 0 & 1 \\
0 & 1 & 0 & 0 & 1 & 1 \\
1 & 1 & 1 & 0 & 1 & 0 \\
\end{array}\right)$
"


✅ All tests completed for type: Int64


In [23]:
test_L_show(Float64)


🔹 Testing Standard Vector:


L"$\left(\begin{array}{r}
1.0 \\
2.0 \\
3.0 \\
\end{array}\right)$
"


🔹 Testing Adjoint Vector (Row Vector):


L"$\left(\begin{array}{r}
1.0 & 2.0 & 3.0 \\
\end{array}\right)$
"


🔹 Testing Standard Matrix:


...


🔹 Testing BlockArray:


L"$\left(\begin{array}{rrr|rrr}
0.0 & 0.0 & 0.0 & 1.0 & 1.0 & 0.0 \\
0.0 & 0.0 & 1.0 & 1.0 & 1.0 & 1.0 \\ \hline
1.0 & 1.0 & 0.0 & 0.0 & 0.0 & 1.0 \\
0.0 & 1.0 & 0.0 & 1.0 & 1.0 & 0.0 \\
1.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 \\
0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 \\
\end{array}\right)$
"


🔹 Testing Adjoint BlockArray:


L"$\left(\begin{array}{rr|rrrr}
0.0 & 0.0 & 1.0 & 0.0 & 1.0 & 0.0 \\
0.0 & 0.0 & 1.0 & 1.0 & 0.0 & 0.0 \\
0.0 & 1.0 & 0.0 & 0.0 & 1.0 & 0.0 \\ \hline
1.0 & 1.0 & 0.0 & 1.0 & 0.0 & 0.0 \\
1.0 & 1.0 & 0.0 & 1.0 & 0.0 & 0.0 \\
0.0 & 1.0 & 1.0 & 0.0 & 0.0 & 1.0 \\
\end{array}\right)$
"


✅ All tests completed for type: Float64


In [24]:
test_L_show(Rational{Int})


🔹 Testing Standard Vector:


L"$\left(\begin{array}{r}
1 \\
2 \\
3 \\
\end{array}\right)$
"


🔹 Testing Adjoint Vector (Row Vector):


L"$\left(\begin{array}{r}
1 & 2 & 3 \\
\end{array}\right)$
"


🔹 Testing Standard Matrix:


...


🔹 Testing BlockArray:


L"$\left(\begin{array}{rrr|rrr}
1 & 1 & 0 & 1 & 1 & 0 \\
0 & 0 & 1 & 1 & 0 & 1 \\ \hline
1 & 0 & 0 & 1 & 1 & 0 \\
1 & 1 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 1 & 0 \\
1 & 0 & 0 & 1 & 1 & 0 \\
\end{array}\right)$
"


🔹 Testing Adjoint BlockArray:


L"$\left(\begin{array}{rr|rrrr}
1 & 0 & 1 & 1 & 1 & 1 \\
1 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 \\ \hline
1 & 1 & 1 & 0 & 0 & 1 \\
1 & 0 & 1 & 0 & 1 & 1 \\
0 & 1 & 0 & 0 & 0 & 0 \\
\end{array}\right)$
"


✅ All tests completed for type: Rational{Int64}


In [25]:
test_L_show(Complex{Float64})


🔹 Testing Standard Vector:


L"$\left(\begin{array}{r}
1.0 \\
2.0 \\
3.0 \\
\end{array}\right)$
"


🔹 Testing Adjoint Vector (Row Vector):


L"$\left(\begin{array}{r}
1.0 & 2.0 & 3.0 \\
\end{array}\right)$
"


🔹 Testing Standard Matrix:


...


🔹 Testing BlockArray:


L"$\left(\begin{array}{rrr|rrr}
0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 1.0 \\
0.0 & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 \\ \hline
1.0 & 1.0 & 1.0 & 1.0 & 1.0 & 0.0 \\
1.0 & 1.0 & 1.0 & 1.0 & 0.0 & 1.0 \\
1.0 & 0.0 & 1.0 & 1.0 & 1.0 & 1.0 \\
1.0 & 1.0 & 1.0 & 1.0 & 0.0 & 1.0 \\
\end{array}\right)$
"


🔹 Testing Adjoint BlockArray:


L"$\left(\begin{array}{rr|rrrr}
0.0 & 0.0 & 1.0 & 1.0 & 1.0 & 1.0 \\
0.0 & 0.0 & 1.0 & 1.0 & 0.0 & 1.0 \\
1.0 & 0.0 & 1.0 & 1.0 & 1.0 & 1.0 \\ \hline
0.0 & 1.0 & 1.0 & 1.0 & 1.0 & 1.0 \\
0.0 & 0.0 & 1.0 & 0.0 & 1.0 & 0.0 \\
1.0 & 0.0 & 0.0 & 1.0 & 1.0 & 1.0 \\
\end{array}\right)$
"


✅ All tests completed for type: ComplexF64


In [26]:
test_L_show(Complex{Rational{Int}})


🔹 Testing Standard Vector:


L"$\left(\begin{array}{r}
1 \\
2 \\
3 \\
\end{array}\right)$
"


🔹 Testing Adjoint Vector (Row Vector):


L"$\left(\begin{array}{r}
1 & 2 & 3 \\
\end{array}\right)$
"


🔹 Testing Standard Matrix:


...


🔹 Testing BlockArray:


L"$\left(\begin{array}{rrr|rrr}
0 & 0 & 0 & 1 & 0 & 0 \\
1 & 0 & 1 & 1 & 0 & 1 \\ \hline
0 & 1 & 0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 & 1 & 1 \\
1 & 0 & 0 & 0 & 0 & 1 \\
1 & 0 & 0 & 0 & 0 & 1 \\
\end{array}\right)$
"


🔹 Testing Adjoint BlockArray:


L"$\left(\begin{array}{rr|rrrr}
0 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 1 & 0 & 1 & 0 & 0 \\ \hline
1 & 1 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 1 & 1 & 1 \\
\end{array}\right)$
"


✅ All tests completed for type: Complex{Rational{Int64}}
